## RacingAI ROS Workshop

Start by opening up the Code editor, and let's look at our initial file structure!

Next, we'll build move into our workspace, build our package, and update our environment.

```
cd ros2_ws
colcon build --symlink-install
source install/setup.bash
```

Now we can run our packages launch file

```
ros2 launch my_bot launch_sim.launch.py
```

Let's have a look at the topics being published

```
ros2 topic list
```

We can get info about a particular topic using

```
ros2 topic info <topic-name>
```

Or listen to it by using

```
ros2 topic echo <topic-name>
```

In another terminal, let's try running our perception Node

```
cd ros2_ws
source install/setup.bash
ros2 run my_bot perception
```

It doesn't do much yet! Let's change that. We can start by installing some dependencies

```
pip3 install -r ~/ros2_ws/src/my_bot/requirements.txt -f https://download.pytorch.org/whl/cpu/torch_stable.html
```

Let's import some of our libraries, and add a basic subscriber to the camera

```
...
from sensor_msgs.msg import Image
...
    def __init__(self):
        ...
        self.subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.listener_callback,
            1)

    def listener_callback(self, msg):
        self.get_logger().info('Received image with size {}x{}'.format(msg.width, msg.height))
...
```

Now if we run it again, we should get some info about the received images!

Let's do some basic parsing and viewing of the images

```
...
import cv2
import cv_bridge
...
    def __init__(self):
        ...
        self.bridge = cv_bridge.CvBridge()
...
    def listener_callback(self, msg):
        ...
        cv_image = self.bridge.imgmsg_to_cv2(
            msg, desired_encoding='passthrough')
        cv_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2BGR)
        cv2.imshow('Image window', cv_image)
        cv2.waitKey(1)
...
```

Now we're ready to start doing some real perception.

In the last workshop, we trained a FasterRCNN model. However, this turned out to be [too slow on machines that don't have a good GPU](https://colab.research.google.com/drive/1PCTSamO_xxDjChp7QYkkyXAT-XDnuQLD?usp=sharing).

[YOLOv8](https://github.com/ultralytics/ultralytics) is the current state of the art, and [has since been trained for cone detection as well](https://colab.research.google.com/drive/1GZZFM5Z363KUtCnkUrznX19M9gGmfTca?usp=sharing). We can download the YOLOv8 model by running this script

```
python3 ~/ros2_ws/src/my_bot/data/pull_cone_detector.py
```

Now let's run it on our images

```
...
from ultralytics import YOLO
...
    def __init__(self):
        super().__init__('perception')
        self.load_model()
    ...
    def listener_callback(self, msg):
        ...
        results = self.model([cv_image])
        res_plotted = results[0].plot()
        cv2.imshow('Image window', res_plotted)
        cv2.waitKey(1)
    
    def load_model(self):
        self.get_logger().info('Loading cone detector model')
        self.model = YOLO("/home/user/ros2_ws/src/my_bot/data/cone-detector-yolov8.pt")
        self.get_logger().info('Model loaded!')
...
```

But when we run our perception node, the inference is really slow. Why?

If we take a look at the processes running using `top`, we can see that Gazebo uses a lot of CPU. Now that we've taken a look at it, let's run it in the background to save CPU.

```
ros2 launch my_bot launch_sim.launch.py gui:=false
```

If we have time remaining, let's have a go at [figuring out how far away the cones are](https://pyimagesearch.com/2015/01/19/find-distance-camera-objectmarker-using-python-opencv/).